In [1]:
import os
import contextlib

import pandas as pd
import seaborn as sns
from datetime import date
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def find_sheet_name(sheet_names):
    cpi_sheet_names = list(
        filter(lambda x: "tongmuc" in x.lower() and "22-23" not in x, sheet_names)
    )

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    cpi_sheet_names = list(filter(lambda x: "tongmuc" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    cpi_sheet_names = list(filter(lambda x: "tong muc" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]


    cpi_sheet_names = list(
        filter(lambda x: "tm" in x.lower() and "xnk" not in x.lower(), sheet_names)
    )

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    if len(sheet_names) == 43:
        return "23-24"

    cpi_sheet_names = list(
        filter(lambda x: "sheet1 (2)" in x.lower(), sheet_names)
    )

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    raise RuntimeError(sheet_names, len(sheet_names))

In [3]:
en_to_vn = {
    "TOTAL": "TỔNG SỐ",
    "Retail sale": "Bán lẻ hàng hóa",
    "Accommodation and catering service": "Dịch vụ lưu trú, ăn uống",
    "Traveling service": "Du lịch lữ hành",
    "Other services": "Dịch vụ khác",
}

In [4]:
def get_tongmuc(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    df = df.dropna(axis=1, how="all")

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "Bán lẻ hàng hóa")
                | (df[df.columns[1]] == "Retail sale ")
                | (df[df.columns[1]] == "Thương nghiệp")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    df[df.columns[0]] = (
        df[df.columns[0]].replace(r"\s+", " ", regex=True).replace(r"\n", " ", regex=True)
    )

    df[df.columns[0]] = df[df.columns[0]].str.strip()

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "TỔNG SỐ")
                    | ((df[df.columns[0]] == "TOTAL"))
                    | ((df[df.columns[0]] == "Tổng số"))
                ].index[0]
            )
        )
    )

    df = df.reset_index(drop=True)

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "Dịch vụ khác")
                    | ((df[df.columns[0]] == "Other services"))
                    | ((df[df.columns[0]] == "Dịch vụ"))
                ].index[0]
                + 1,
                len(df.index),
            )
        )
    )

    df = df.dropna(axis=1, how="all")

    if len(df[df.columns[0]][df[df.columns[0]] == "TOTAL"].index) > 0:
        df[df.columns[0]] = df[df.columns[0]].map(en_to_vn)

    if len(df.columns) == 7:
        df = df.drop(columns=[df.columns[1], df.columns[4]])
    elif len(df.columns) == 4 and int(date.split("-")[1]) == 1:
        df = df.drop(columns=[df.columns[2]])
        df["A"] = df[df.columns[1]]
        df["B"] = df[df.columns[2]]
        df = df[[df.columns[0], df.columns[1], "A", df.columns[2], "B"]]
    elif len(df.columns) == 5 and int(date.split("-")[1]) == 1:
        df = df.drop(columns=[df.columns[1], df.columns[3]])
        df["A"] = df[df.columns[1]]
        df["B"] = df[df.columns[2]]
        df = df[[df.columns[0], df.columns[1], "A", df.columns[2], "B"]]
    elif len(df.columns) == 5:
        df = df.drop(columns=[df.columns[3]])

    if len(df.columns) == 5:
        df.columns = ["Phân Loại", "M", "YTD", "M-YOY", "YTD-YOY"]

        df = df[pd.to_numeric(df["M"], errors="coerce").notnull()]
        df = df[pd.to_numeric(df["YTD"], errors="coerce").notnull()]
        df = df[pd.to_numeric(df["M-YOY"], errors="coerce").notnull()]
        df = df[pd.to_numeric(df["YTD-YOY"], errors="coerce").notnull()]
    elif len(df.columns) == 4:
        df.columns = ["Phân Loại", "M", "YTD", "YTD-YOY"]

        df = df[pd.to_numeric(df["M"], errors="coerce").notnull()]
        df = df[pd.to_numeric(df["YTD"], errors="coerce").notnull()]
        df = df[pd.to_numeric(df["YTD-YOY"], errors="coerce").notnull()]

    df["Date"] = date
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)

    return df


xl = pd.ExcelFile("../datas/2019-03-29-02-Bieu-3-2019-1.xlsx")

sheet_name = find_sheet_name(xl.sheet_names)

get_tongmuc(xl, sheet_name, "2019-03-29")

,Phân Loại,M,YTD,YTD-YOY,Date
0,TỔNG SỐ,392189.659427,1184880.729411,111.962053,2019-03-29
1,Bán lẻ hàng hóa,301021.230251,910383.216096,113.391902,2019-03-29
2,"Dịch vụ lưu trú, ăn uống",46675.114289,140028.721503,109.236668,2019-03-29
3,Du lịch lữ hành,3508.75746,11303.62201,112.840835,2019-03-29
4,Dịch vụ khác,40984.557428,123165.169802,105.07389,2019-03-29


In [5]:
import os
import contextlib

dfs_tongmuc_origin = []
for file_name in sorted(
    os.listdir("../datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    if file_name.startswith(".~"):
        continue

    try:
        file_path = os.path.join("../datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2012:
            continue

        xl = pd.ExcelFile(file_path)

        df_tongmuc = get_tongmuc(xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}")

        dfs_tongmuc_origin.append(df_tongmuc)
    except Exception as e:
        print(file_name)
        raise e

In [12]:
dfs_tongmuc = pd.concat(dfs_tongmuc_origin, ignore_index=True, axis=0)
dfs_tongmuc["Date"] = pd.to_datetime(dfs_tongmuc["Date"], dayfirst=False)
dfs_tongmuc

,Phân Loại,M,YTD,M-YOY,YTD-YOY,Date
0,TỔNG SỐ,519760.357083,2580175.221234,109.48295,108.652786,2024-05-29
1,Bán lẻ hàng hóa,402373.762424,1998188.838348,108.198967,107.419514,2024-05-29
2,"Dịch vụ lưu trú, ăn uống",60519.268902,296269.806656,116.955363,115.061725,2024-05-29
3,Du lịch lữ hành,4976.8,24246.379361,134.304033,145.105638,2024-05-29
4,Dịch vụ khác,51890.471207,261470.196868,109.458657,108.797458,2024-05-29
...,...,...,...,...,...,...
923,Khu vực có vốn đầu tư nước ngoài,5617,5617,135.36,135.36,2012-01-30
924,Thương nghiệp,149653,149653,121.23,121.23,2012-01-30
925,"Khách sạn, nhà hàng",21991,21991,133.33,133.33,2012-01-30
926,Du lịch,1362,1362,111.1,111.1,2012-01-30


In [13]:
convert_name = {
    "TỔNG SỐ": "TỔNG SỐ",
    "Bán lẻ hàng hóa": "Bán lẻ hàng hóa",
    "Dịch vụ lưu trú, ăn uống": "Dịch vụ lưu trú, ăn uống",
    "Du lịch lữ hành": "Du lịch lữ hành",
    "Dịch vụ khác": "Dịch vụ khác",
    "Nhà nước": "Nhà nước",
    "Ngoài nhà nước": "Ngoài nhà nước",
    "Khu vực có vốn đầu tư nước ngoài": "Khu vực có vốn đầu tư nước ngoài",
    "Tổng số": "TỔNG SỐ",
    "Thương nghiệp": "Bán lẻ hàng hóa",
    "Khách sạn, nhà hàng": "Dịch vụ lưu trú, ăn uống",
    "Du lịch": "Du lịch lữ hành",
    "Dịch vụ": "Dịch vụ khác",
    "Tập thể": "Tập thể",
    "Tư nhân": "Tư nhân",
    "Cá thể": "Cá thể",
}

for pl in dfs_tongmuc["Phân Loại"].unique().tolist():
    assert pl in convert_name.keys(), pl

dfs_tongmuc["Phân Loại"] = dfs_tongmuc["Phân Loại"].map(convert_name)
dfs_tongmuc

,Phân Loại,M,YTD,M-YOY,YTD-YOY,Date
0,TỔNG SỐ,519760.357083,2580175.221234,109.48295,108.652786,2024-05-29
1,Bán lẻ hàng hóa,402373.762424,1998188.838348,108.198967,107.419514,2024-05-29
2,"Dịch vụ lưu trú, ăn uống",60519.268902,296269.806656,116.955363,115.061725,2024-05-29
3,Du lịch lữ hành,4976.8,24246.379361,134.304033,145.105638,2024-05-29
4,Dịch vụ khác,51890.471207,261470.196868,109.458657,108.797458,2024-05-29
...,...,...,...,...,...,...
923,Khu vực có vốn đầu tư nước ngoài,5617,5617,135.36,135.36,2012-01-30
924,Bán lẻ hàng hóa,149653,149653,121.23,121.23,2012-01-30
925,"Dịch vụ lưu trú, ăn uống",21991,21991,133.33,133.33,2012-01-30
926,Du lịch lữ hành,1362,1362,111.1,111.1,2012-01-30


In [14]:
dfs_tongmuc.to_csv('../csv/tongmuc.csv')

In [15]:
# dfs_tongmuc.loc[dfs_tongmuc["M-YOY"] < 30, "M-YOY"] = (
#     dfs_tongmuc.loc[dfs_tongmuc["M-YOY"] < 30, "M-YOY"] + 100
# )
# dfs_tongmuc.loc[dfs_tongmuc["YTD-YOY"] < 30, "YTD-YOY"] = (
#     dfs_tongmuc.loc[dfs_tongmuc["YTD-YOY"] < 30, "YTD-YOY"] + 100
# )

# dfs_tongmuc["M-YOY"] = dfs_tongmuc["M-YOY"] - 100
# dfs_tongmuc["YTD-YOY"] = dfs_tongmuc["YTD-YOY"] - 100

In [16]:
# nganh = "TỔNG SỐ"

# dfs_tongmuc_hanghoa = dfs_tongmuc[dfs_tongmuc["Phân Loại"] == nganh]
# dfs_tongmuc_hanghoa["PCT_YOY"] = dfs_tongmuc_hanghoa["M"].pct_change(-12)
# dfs_tongmuc["M-YOY-PCT"] = dfs_tongmuc_hanghoa["PCT_YOY"]

In [17]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_tongmuc[
    (
        (dfs_tongmuc["Phân Loại"] == "TỔNG SỐ")
        | (dfs_tongmuc["Phân Loại"] == "Bán lẻ hàng hóa")
        | (dfs_tongmuc["Phân Loại"] == "Dịch vụ lưu trú, ăn uống")
        # | (dfs_tongmuc["Phân Loại"] == "Du lịch lữ hành")
        # | (dfs_tongmuc["Phân Loại"] == "Dịch vụ khác")
    )
    # & (dfs_tongmuc["Date"] > "2021-01-01")
]

fig = px.area(df_show, x="Date", y="M", color="Phân Loại")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()